In [1]:
def append_composite_dictionary(composite_dict, models):
    for model in models:
        composite_dict.update(model.param_dict)

In [2]:
def update_component_dictionaries(composite_dict, models):
    for model in models:
        for key in model.param_dict.keys():
            model.param_dict[key] = composite_dict[key]

In [3]:
class AssemblyBias(object):
    def __init__(self, sample_name):
        self.param_dict = {}
        self.sample_name = sample_name
        self.modelname = 'assembly_bias_model_'+self.sample_name
        self.param_dict['assembly_bias_param0_'+self.sample_name] = 1.

    def assembly_bias_strength(self):
        return self.param_dict['assembly_bias_param0_'+self.sample_name]

In [14]:
class Galaxies(object):
    def __init__(self, sample_name, **kwargs):
        
        self.sample_name = sample_name
        self.param_dict = {}
        self.param_dict['nz_param0_'+self.sample_name] = 0.1
        self.param_dict['nz_param1_'+self.sample_name] = 0.2
        self.param_dict['large_scale_bias_'+self.sample_name] = 1.5

        try:
            self.models = kwargs['models']
        except KeyError:
            self.models = []
            
        for model in self.models:
            setattr(self, model.modelname, model)
        append_composite_dictionary(self.param_dict, self.models)
    
    def true_redshift_pdf(self, z):
        for model in self.models:
            #  Propagate parameters from composite model down to component model
            for key in model.param_dict.keys():
                model.param_dict[key] = self.param_dict[key]
                
        return self.param_dict['nz_param0'] + self.param_dict['nz_param0'] + z
    
    def large_scale_bias(self):
        update_component_dictionaries(self.param_dict, self.models)
        
        modelname = 'assembly_bias_model_' + self.sample_name
        try:
            assembias_model = getattr(self, modelname)
            assembias_value = assembias_model.assembly_bias_strength()
        except AttributeError:
            #  maybe this sample has no assembly bias
            assembias_value = 0.0

        return self.param_dict['large_scale_bias_'+self.sample_name] + assembias_value

In [15]:
assembias_blue = AssemblyBias('blue')
blue_galaxies = Galaxies('blue', models=[assembias_blue])

assembias_red = AssemblyBias('red')
red_galaxies = Galaxies('red', models=[assembias_red])

In [16]:
red_galaxies.param_dict

{'assembly_bias_param0_red': 1.0,
 'large_scale_bias_red': 1.5,
 'nz_param0_red': 0.1,
 'nz_param1_red': 0.2}

In [17]:
print(blue_galaxies.large_scale_bias())
blue_galaxies.param_dict['assembly_bias_param0_blue'] = 3.5
print(blue_galaxies.large_scale_bias())


2.5
5.0


In [7]:
class GalaxyGalaxyTheoryVector(object):
    def __init__(self, galaxies1, galaxies2, **kwargs):
        
        try:
            self.models = kwargs['models']
        except KeyError:
            self.models = []

        self.param_dict = {}
        append_composite_dictionary(self.param_dict, self.models)
        append_composite_dictionary(self.param_dict, [galaxies1, galaxies2])
        self.galaxies1 = galaxies1
        self.galaxies2 = galaxies2
        
    def large_scale_bias(self):
        update_component_dictionaries(self.param_dict, [self.galaxies1, self.galaxies2])
        return np.sqrt(self.galaxies1.large_scale_bias()*self.galaxies2.large_scale_bias())

In [8]:
gg = GalaxyGalaxyTheoryVector(blue_galaxies, red_galaxies)

In [9]:
gg.param_dict

{'assembly_bias_param0_blue': 3.5,
 'assembly_bias_param0_red': 1.0,
 'large_scale_bias_blue': 1.5,
 'large_scale_bias_red': 1.5,
 'nz_param0_blue': 0.1,
 'nz_param0_red': 0.1,
 'nz_param1_blue': 0.2,
 'nz_param1_red': 0.2}

In [10]:
gg.large_scale_bias()

3.5355339059327378

In [11]:
gg.param_dict['large_scale_bias_red'] = 3.4
gg.large_scale_bias()

4.6904157598234297

In [12]:
gg.param_dict['assembly_bias_param0_red'] = 3.4
gg.large_scale_bias()

5.8309518948453007